In [ ]:
import pandas as pd

In [ ]:
test = pd.read_csv("/content/Data_test.csv")
train = pd.read_csv("/content/Data_train.csv")

In [ ]:
test.info()
test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   type                300 non-null    object
 1   group               288 non-null    object
 2   education           300 non-null    object
 3   meal                300 non-null    object
 4   preparation course  300 non-null    object
dtypes: object(5)
memory usage: 11.8+ KB


type                   0
group                 12
education              0
meal                   0
preparation course     0
dtype: int64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   type                700 non-null    object
 1   group               680 non-null    object
 2   education           700 non-null    object
 3   meal                700 non-null    object
 4   preparation course  700 non-null    object
 5   score-1             700 non-null    int64 
 6   score-2             700 non-null    int64 
 7   score-3             700 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 43.9+ KB


In [ ]:
train['group'].fillna('Unknown', inplace=True)
test['group'].fillna('Unknown', inplace=True)


In [ ]:
train['Pass'] = train.apply(lambda x: 1 if x["score-1"] > 50 and x["score-2"] > 50 and x["score-3"] > 50 else 0, axis=1)

In [ ]:
train['Pass'].value_counts()

Pass
1    555
0    145
Name: count, dtype: int64

In [ ]:
wild = train[train['type'] == 'wild']

In [ ]:
wild[wild['Pass'] == 1].shape[0] / wild.shape[0]

0.7813411078717201

In [ ]:
dom = train[train['type'] != 'wild']

In [ ]:
dom[dom['Pass'] == 1].shape[0] / dom.shape[0]

0.803921568627451

In [ ]:
0.803921568627451 - 0.7813411078717201

0.022580460755730947

In [ ]:
train[train['Pass'] == 0]['education'].value_counts()

education
some high school      42
high school           35
some college          28
associate's degree    22
bachelor's degree     12
master's degree        6
Name: count, dtype: int64

In [ ]:
wild[(wild['Pass'] == 1)]['preparation course'].value_counts()

preparation course
none         152
completed    116
Name: count, dtype: int64

In [ ]:
train['score-1'].median()

66.0

In [ ]:
import numpy as np
q3, q1 = np.percentile(train['score-1'], [75 ,25], method='lower')
iqr = q3 - q1

In [ ]:
iqr

21

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_train = train.drop(columns=['Pass', 'score-1', 'score-2', 'score-3'])
drop_enc = OneHotEncoder(drop='first').fit(X_train)

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True, dtype='int')

In [ ]:
X_test = pd.get_dummies(test, drop_first=True, dtype='int')

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, splitter='best')
res = cross_val_score(clf, X_train, train['Pass'], cv=3, scoring='roc_auc')

In [ ]:
res.mean()

0.6669447508733223

In [ ]:
from sklearn.model_selection import GridSearchCV
res = GridSearchCV(clf, param_grid={'max_depth': [i for i in range(2, 21)]}, cv=3, scoring='roc_auc')

In [ ]:
res.fit(X_train, train['Pass'])

GridSearchCV(cv=3,
             estimator=DecisionTreeClassifier(criterion='entropy', max_depth=5),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16, 17, 18, 19, 20]},
             scoring='roc_auc')

In [ ]:
res.best_params_

{'max_depth': 2}

In [ ]:
import random
clf = DecisionTreeClassifier(criterion='entropy', max_depth=2, splitter='best')
clf.fit(X_train, train['Pass'])
res = clf.predict(X_test)
with open("res.txt", mode="w", encoding="utf-8") as fout:
  for el in res:
    print(random.choice([0, 1, 1]))
    fout.write("{}\n".format(el))

1
1
1
1
1
0
1
1
0
1
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
0
1
1
0
0
0
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
0
1
0
1
1
0
1
1
1
0
1
1
1
1
0
1
1
1
1
0
1
1
0
1
1
0
1
1
1
1
1
1
0
1
1
1
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
0
0
1
0
1
0
1
1
1
0
1
1
1
1
0
0
1
1
0
1
1
1
1
0
0
0
0
0
1
0
1
1
1
1
1
0
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
1
1
0
1
0
1
1
0
0
0
0
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
0
1
1
0
1
1
1
0
1
0
1
1
1
1
0
1
1
0
1
1
1
0
0
1
0
0
1
0
1
1
0
1
1
1
1
1
1
0
0
1
0
0
1
0
0
1
1
1
1
1
1
1
0
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
0
0
1
0
1
0
0
0
1
1
1
0
1


In [ ]:
train['cat_bio'] = train.apply(lambda row: '({}, {})'.format(row['type'], row['group']), axis=1)
test['cat_bio'] = test.apply(lambda row: '({}, {})'.format(row['type'], row['group']), axis=1)

In [ ]:
train['cat_bio']

0          (wild, group B)
1      (domestic, group C)
2      (domestic, group C)
3      (domestic, group B)
4          (wild, group E)
              ...         
695        (wild, group C)
696    (domestic, group B)
697        (wild, group E)
698        (wild, group E)
699        (wild, group C)
Name: cat_bio, Length: 700, dtype: object

In [ ]:
test.drop(columns=['type', 'group'], inplace=True)

In [ ]:
X_train = pd.get_dummies(train, drop_first=True, dtype='int')
X_train.drop(columns=['Pass', 'score-1', 'score-2', 'score-3'], inplace=True)
X_test = pd.get_dummies(test, drop_first=True, dtype='int')

In [ ]:
clf = DecisionTreeClassifier(max_depth=5, criterion='entropy')
clf.fit(X_train, train["Pass"])

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(train["Pass"], clf.predict(X_train))

0.5812364088226157

In [ ]:
pred = clf.predict(X_test)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- group_group A
- group_group B
- group_group C
- group_group D
- group_group E
- ...
